### Derivatives Final Project_Team 2

For Excel we need the following:
1. A sheet with portfolio weights (Ticker, Weight)
2. One or more sheets with historical price data for the tickers (e.g. daily adjusted close prices over 1 year)

In [ ]:
import pandas as pd
import numpy as np

# Load Excel file
file_path = "portfolio.xlsx"
portfolio_df = pd.read_excel(file_path, sheet_name='Portfolio')
tickers = portfolio_df['Ticker'].tolist()
weights = portfolio_df['Weight'].values
weights /= weights.sum()

# Load price data from each sheet into a single DataFrame
price_data = pd.DataFrame()
for ticker in tickers:
    df = pd.read_excel(file_path, sheet_name=ticker)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    price_data[ticker] = df['Adj Close']

# Calculate daily returns
returns = price_data.pct_change().dropna()

# Annualized stats
mu = returns.mean() * 252
cov = returns.cov() * 252

print("Tickers:", tickers)
print("Weights:", weights)
print("Mean returns:\n", mu)

In [ ]:
T = 1
dt = 1/252
N = 1000
model_returns = {}

for model in ['gbm', 'merton', 'cev', 'heston']:
    sim_returns = []

    for i, ticker in enumerate(tickers):
        S0 = price_data[ticker].iloc[-1]
        sigma_i = returns[ticker].std() * np.sqrt(252)
        mu_i = mu[ticker]

        if model == 'gbm':
            path = simulate_gbm(S0, mu_i, sigma_i, T, dt, N)
        elif model == 'merton':
            path = simulate_merton(S0, mu_i, sigma_i, lamb=0.75, mu_j=-0.02, sigma_j=0.05, T=T, dt=dt, N=N)
        elif model == 'cev':
            path = simulate_cev(S0, mu_i, sigma_i, beta=0.8, T=T, dt=dt, N=N)
        elif model == 'heston':
            path = simulate_heston(S0, V0=sigma_i**2, mu=mu_i, kappa=2.0, theta=sigma_i**2, xi=0.1, rho=-0.7, T=T, dt=dt, N=N)

        ret = (path[-1] - S0) / S0
        sim_returns.append(ret)

    sim_returns = np.array(sim_returns)
    port_returns = np.dot(weights, sim_returns)
    model_returns[model] = port_returns


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
for model, returns in model_returns.items():
    plt.hist(returns, bins=50, alpha=0.6, label=model)
plt.legend()
plt.title("Portfolio Simulated Returns by Model")
plt.xlabel("Return")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()